In [19]:
import cv2
import numpy as np

In [23]:
img = cv2.imread(r"C:\Users\SRI\Downloads\WhatsApp Image 2020-08-20 at 22.25.25.jpeg",0)
img = cv2.medianBlur(img,5)
cimg = cv2.cvtColor(img,cv2.COLOR_GRAY2BGR)


In [24]:
circles = cv2.HoughCircles(img,cv2.HOUGH_GRADIENT,1,20,
                            param1=50,param2=30,minRadius=0,maxRadius=0)

In [25]:
circles = np.uint16(np.around(circles))
for i in circles[0,:]:
    # draw the outer circle
    cv2.circle(cimg,(i[0],i[1]),i[2],(0,255,0),2)
    # draw the center of the circle
    cv2.circle(cimg,(i[0],i[1]),2,(0,0,255),3)

In [26]:
cv2.imshow('detected circles',cimg)
cv2.waitKey(0)

cv2.destroyAllWindows()

In [35]:
import cv2
img = cv2.imread("beforethreshold..jpg")
print(img.shape)
cv2.imshow("img",img)
cv2.waitKey(0)

#cv2.destroyAllwindows()
#img.shape

(478, 450, 3)


-1

In [36]:
crop_img = img[20:100,20:100]
cv2.imshow("cropped", crop_img)
cv2.waitKey(0)
cv2.destroyAllWindows()

<class 'int'>


In [2]:
from math import sqrt, atan2, pi
import numpy as np

def canny_edge_detector(input_image):
    input_pixels = input_image.load()
    width = input_image.width
    height = input_image.height

    # Transform the image to grayscale
    grayscaled = compute_grayscale(input_pixels, width, height)

    # Blur it to remove noise
    blurred = compute_blur(grayscaled, width, height)

    # Compute the gradient
    gradient, direction = compute_gradient(blurred, width, height)

    # Non-maximum suppression
    filter_out_non_maximum(gradient, direction, width, height)

    # Filter out some edges
    keep = filter_strong_edges(gradient, width, height, 20, 25)

    return keep


def compute_grayscale(input_pixels, width, height):
    grayscale = np.empty((width, height))
    for x in range(width):
        for y in range(height):
            pixel = input_pixels[x, y]
            grayscale[x, y] = (pixel[0] + pixel[1] + pixel[2]) / 3
    return grayscale


def compute_blur(input_pixels, width, height):
    # Keep coordinate inside image
    clip = lambda x, l, u: l if x < l else u if x > u else x

    # Gaussian kernel
    kernel = np.array([
        [1 / 256,  4 / 256,  6 / 256,  4 / 256, 1 / 256],
        [4 / 256, 16 / 256, 24 / 256, 16 / 256, 4 / 256],
        [6 / 256, 24 / 256, 36 / 256, 24 / 256, 6 / 256],
        [4 / 256, 16 / 256, 24 / 256, 16 / 256, 4 / 256],
        [1 / 256,  4 / 256,  6 / 256,  4 / 256, 1 / 256]
    ])

    # Middle of the kernel
    offset = len(kernel) // 2

    # Compute the blurred image
    blurred = np.empty((width, height))
    for x in range(width):
        for y in range(height):
            acc = 0
            for a in range(len(kernel)):
                for b in range(len(kernel)):
                    xn = clip(x + a - offset, 0, width - 1)
                    yn = clip(y + b - offset, 0, height - 1)
                    acc += input_pixels[xn, yn] * kernel[a, b]
            blurred[x, y] = int(acc)
    return blurred


def compute_gradient(input_pixels, width, height):
    gradient = np.zeros((width, height))
    direction = np.zeros((width, height))
    for x in range(width):
        for y in range(height):
            if 0 < x < width - 1 and 0 < y < height - 1:
                magx = input_pixels[x + 1, y] - input_pixels[x - 1, y]
                magy = input_pixels[x, y + 1] - input_pixels[x, y - 1]
                gradient[x, y] = sqrt(magx**2 + magy**2)
                direction[x, y] = atan2(magy, magx)
    return gradient, direction


def filter_out_non_maximum(gradient, direction, width, height):
    for x in range(1, width - 1):
        for y in range(1, height - 1):
            angle = direction[x, y] if direction[x, y] >= 0 else direction[x, y] + pi
            rangle = round(angle / (pi / 4))
            mag = gradient[x, y]
            if ((rangle == 0 or rangle == 4) and (gradient[x - 1, y] > mag or gradient[x + 1, y] > mag)
                    or (rangle == 1 and (gradient[x - 1, y - 1] > mag or gradient[x + 1, y + 1] > mag))
                    or (rangle == 2 and (gradient[x, y - 1] > mag or gradient[x, y + 1] > mag))
                    or (rangle == 3 and (gradient[x + 1, y - 1] > mag or gradient[x - 1, y + 1] > mag))):
                gradient[x, y] = 0


def filter_strong_edges(gradient, width, height, low, high):
    # Keep strong edges
    keep = set()
    for x in range(width):
        for y in range(height):
            if gradient[x, y] > high:
                keep.add((x, y))

    # Keep weak edges next to a pixel to keep
    lastiter = keep
    while lastiter:
        newkeep = set()
        for x, y in lastiter:
            for a, b in ((-1, -1), (-1, 0), (-1, 1), (0, -1), (0, 1), (1, -1), (1, 0), (1, 1)):
                if gradient[x + a, y + b] > low and (x+a, y+b) not in keep:
                    newkeep.add((x+a, y+b))
        keep.update(newkeep)
        lastiter = newkeep

    return list(keep)




In [5]:
from PIL import Image, ImageDraw
from math import sqrt, pi, cos, sin
#from canny import canny_edge_detector
from collections import defaultdict
import cv2

# Load image:
input_image = Image.open(r"C:\Users\SRI\Documents\OpenCVImageProcess\WhatsApp Image 2020-08-20 at 22.25.25.jpeg")

# Output image:
output_image = Image.new("RGB", input_image.size)
output_image.paste(input_image)
draw_result = ImageDraw.Draw(output_image)

# Find circles
rmin = 10
rmax = 12
steps = 100
threshold = 0.6

points = []
for r in range(rmin, rmax + 1):
    for t in range(steps):
        points.append((r, int(r * cos(2 * pi * t / steps)), int(r * sin(2 * pi * t / steps))))

acc = defaultdict(int)
for x, y in canny_edge_detector(input_image):
    for r, dx, dy in points:
        a = x - dx
        b = y - dy
        acc[(a, b, r)] += 1

circles = []
for k, v in sorted(acc.items(), key=lambda i: -i[1]):
    x, y, r = k
    if v / steps >= threshold and all((x - xc) ** 2 + (y - yc) ** 2 > rc ** 2 for xc, yc, rc in circles):
        print(v / steps, x, y, r)
        circles.append((x, y, r))

for x, y, r in circles:
    #cv2.putText(img, "centroid", (cX - 25, cY - 25),cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2)
    
    draw_result.ellipse((x-r, y-r, x+r, y+r), outline=(255,0,0,0))
    #img2=cv2.imread(r"C:\Users\SRI\Documents\OpenCVImageProcess\WhatsApp Image 2020-08-20 at 22.25.25.jpeg")
    crop_img=input_image[x-r+5,y-r+5:x+r+5,y+r+5]
    cv2.imwrite("img"+str(x)+".jpeg",crop_img)
    #cv2.imshow("img",crop_img)
    #cv2.waitKey(0)
# Save output image
print(x-r+5)
print(y-r+5)
print(x+r+5)
print(y+r+5)
#crop_img=img2[x-r:y-r,x+r:y+r]
#cv2.imwrite("crop_img.jpg",crop_img)

#cv2.imshow("img",crop_img)
output_image.save("newimg_10_12_100_06.jpeg")

0.92 666 345 10
0.92 615 345 10
0.92 306 565 10
0.92 197 289 10
0.92 73 372 10
0.92 279 395 10
0.92 640 457 11
0.92 708 373 10
0.9 355 92 10
0.89 542 464 11
0.88 246 92 10
0.88 667 473 10
0.88 677 216 10
0.87 354 130 10
0.87 79 448 10
0.87 796 559 10
0.87 432 47 10
0.86 637 113 10
0.84 174 365 10
0.84 220 105 10
0.83 535 277 10
0.83 694 657 10
0.82 362 435 10
0.81 574 481 10
0.79 741 344 10
0.79 274 313 10
0.78 802 382 10
0.78 28 580 10
0.78 542 188 10
0.77 263 367 10
0.76 192 205 10
0.76 306 60 10
0.75 780 344 10
0.75 222 544 10
0.74 26 456 10
0.74 655 141 10
0.74 432 76 10
0.73 719 311 10
0.73 428 194 10
0.72 341 409 10
0.71 66 562 10
0.7 243 394 10
0.7 508 277 10
0.69 222 484 10
0.68 552 161 10
0.68 340 281 10
0.67 65 538 11
0.66 212 363 11
0.66 430 118 11
0.65 783 87 10
0.64 336 61 10
0.64 612 161 10
0.64 287 91 10
0.63 27 521 10
0.63 396 96 10
0.63 273 250 10
0.63 273 532 10
0.63 432 149 10
0.62 463 91 10
0.62 222 530 11
0.62 691 308 10
0.6 726 645 11
0.6 226 501 10
0.6 222 515 10

TypeError: 'JpegImageFile' object is not subscriptable

In [18]:
img=r"C:\Users\SRI\Documents\OpenCVImageProcess\WhatsApp Image 2020-08-20 at 22.25.25.jpeg"
gray_image = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
#gray = cv2.cvtColor(cv2.UMat(imgUMat), cv2.COLOR_RGB2GRAY)

# convert the grayscale image to binary image

ret,thresh = cv2.threshold(gray_image,127,255,0)

 

# calculate moments of binary image

M = cv2.moments(thresh)

 

# calculate x,y coordinate of center

cX = int(M["m10"] / M["m00"])

cY = int(M["m01"] / M["m00"])

# put text and highlight the center

cv2.circle(img, (cX, cY), 5, (255, 255, 255), -1)

cv2.putText(img, "centroid", (cX - 25, cY - 25),cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2)

# display the image

cv2.imshow("Image", img)

cv2.waitKey(0)


NameError: name 'imgUMat' is not defined